# EDA
Como paso inicial se importa las librerias a utilizar en nuestro proceso de exploración de los datos que se encuentran comprimidos en formato .gz; estos datos vienen en un formato json, se va a convertirlos en data frames haciendo uso de pandas para ser guardados en un formato csv.

In [1]:
# *** HENRY ***
# *** PROYECTO INDIVIDUAL 01 ***
# *** Sistema de Recomendación de Videojuegos ***
# *** MLOps Engineer ***

# ARCHIVO 'users_items.json.gz'

# LIBRERÍAS
import pandas as pd
import gzip
import ast
import json

data = []
with gzip.open("C:\\Users\\PC\\Music\\Programas_VSC\\HENRY_PI_01\\users_items.json.gz", 'rb') as f:
  for line in f:
    data.append(ast.literal_eval(line.decode('utf-8')))
items = pd.DataFrame(data)
f.close()

In [2]:
# Mostrando las 5 primeras filas

items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [3]:
# Desempaquetado de columna items
items_data = []
for index, row in items.iterrows():
    user_id = row['user_id']
    user_url = row['user_url']
    if 'items' in row and isinstance(row['items'], list):
        items_data.extend([{'user_id': user_id, 'user_url': user_url, **item_value} for item_value in row['items']])
items = pd.DataFrame(items_data)

In [4]:
# Mostrando las 5 primeras filas luego de desempaquetar la columna anidada items
items.head()

,user_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6,0
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0,0
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7,0
3,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40,Deathmatch Classic,0,0
4,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50,Half-Life: Opposing Force,0,0


In [5]:
# Obteniendo informacion acerca del dataframe
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 6 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   user_id           object
 1   user_url          object
 2   item_id           object
 3   item_name         object
 4   playtime_forever  int64 
 5   playtime_2weeks   int64 
dtypes: int64(2), object(4)
memory usage: 235.9+ MB


##### Descripción del DataFrame:
Se puede observar que el DataFrame tiene 6 columnas y 5153209 filas.
También se puede notar que no existen filas donde todas las columnas son NaN y que se pueden borrar.
Explorando las columnas nos podemos dar cuenta que para nuestro objetivo no es necesario la columna 'playtime_2weeks', por lo que se pueden borrar.

Por otro lado, se hace una pequeña descripción de cada una de las columnas:
- user_id: es un identificador único par acada usuario.
- items_url: es el URL del perfil de cada usuario.
- item_id: es el identificador del juego.
- item_name: es el nomvre del contenido del juego.
- playtime_forever: es el tiempo acumulado que un usuario ha jugado.
- playtime_2weeks: es el tiempo acumulado que un usuario ha jugado en las últimas dos semanas

# ETL
Como segundo paso se procede a realizar las transformaciones tomando en cuenta el análisis exploratorio de los datos.

In [6]:
# Cambiando el tipo de dato de object a integer en la columna 'item_id'

items['item_id'] = items['item_id'].astype(int)

In [7]:
# Muestra las filas duplicadas
items.loc[items.duplicated()]

,user_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
164294,Nikiad,http://steamcommunity.com/id/Nikiad,20,Team Fortress Classic,5,0
164295,Nikiad,http://steamcommunity.com/id/Nikiad,50,Half-Life: Opposing Force,0,0
164296,Nikiad,http://steamcommunity.com/id/Nikiad,70,Half-Life,0,0
164297,Nikiad,http://steamcommunity.com/id/Nikiad,130,Half-Life: Blue Shift,0,0
164298,Nikiad,http://steamcommunity.com/id/Nikiad,220,Half-Life 2,198,0
...,...,...,...,...,...,...
4898223,76561198080057659,http://steamcommunity.com/profiles/76561198080...,213670,South Park™: The Stick of Truth™,725,0
4898224,76561198080057659,http://steamcommunity.com/profiles/76561198080...,221910,The Stanley Parable,53,0
4898225,76561198080057659,http://steamcommunity.com/profiles/76561198080...,261030,The Walking Dead: Season Two,253,0
4898226,76561198080057659,http://steamcommunity.com/profiles/76561198080...,273110,Counter-Strike Nexon: Zombies,0,0


In [8]:
# Eliminando las filas duplicadas
items = items.drop_duplicates()

In [9]:
# Eliminando columnas que no intervienen en nuestro análisis

items.drop( ['playtime_2weeks'], axis=1, inplace=True )

In [6]:
# Guardabndo el archivo limpio en formato csv

items.to_csv('items_limpio.csv', index=False)
items.to_parquet('items_limpia.parquet')